In [1]:
import pandas as pd
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import glob
import os
import xlrd
import math
from decimal import *
import datetime
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
from Elia_ActivatedEnergyVolumesPrices_datareadin import read_Elia_ActivEnergyVol, read_Elia_ActivEnergyPrice,FetchAndCombine_Elia_ActivEnergy,combine_Elia_ActivEnergy
from Elia_R2plus_activationV2 import R2plusSingleOptimum, R2plusMultiTimeOptimum

ModuleNotFoundError: No module named 'Elia_ActivatedEnergyVolumesPrices_datareadin'

In [ ]:
import six

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#suppressing some warnings
pd.options.mode.chained_assignment = None

In [ ]:
#store the combined data as a Pickle (.pkl) file with the filename including the date and hour of now.
picklename = "..\data\AncilServR1R2_" + pd.datetime.now().strftime('%Y_%m_%d_%H_%M_%S') + ".pkl"
df.to_pickle(picklename)

In [ ]:
#Get the data from Elia (internet connection needed):
df=FetchAndCombine_Elia_ActivEnergy("12-2017","01-2018","all")
#store for later usage in a Pickle dataformat:
#picklename = "..\data\ActivEnergyPriceVol_01-2012_12-2017.pkl"
#df.to_pickle(picklename)

In [ ]:
#Read-in the Elia-data stored in Pickle format
df = pd.read_pickle("../data/ActivEnergyPriceVol_01-2012_12-2017.pkl")

In [3]:
#get the necessary dataframes (from stored pickle files) and use the R2plusSingleOptimum function
dfAct = pd.read_pickle("../data/ActivEnergyPriceVol_01-2012_12-2017.pkl")
dfCap = pd.read_pickle("../data/AncilServR1R2_2014-2017.pkl")
dfsingle=R2plusSingleOptimum("2017-01-02","2017-01-08",1,dfAct,dfCap,True,True)

In [34]:
#get the necessary dataframes (from stored pickle files) and use the R2plusMultiTimeOptimum function
dfAct = pd.read_pickle("../data/ActivEnergyPriceVol_01-2012_12-2017.pkl")
dfCap = pd.read_pickle("../data/AncilServR1R2_2014-2017.pkl")
dfmulti=R2plusMultiTimeOptimum("2017-01-02","2017-12-31",7,20,dfAct,dfCap,True,True)

In [58]:
#MWh extra produced electricity due to R2+
PowerExtra=dfmulti.groupby(["timestamp"])[["volumeActiv"]].max().sum()*20*168
#calculate cost of extra gas consumption, based on average price of gas spot market
int(PowerExtra*2*20)

1313655

In [60]:
int(PowerExtra)

32841

In [35]:
yearlyremunAct=dfmulti.groupby(["timestamp"])[["returnAct"]].max().sum()
yearlyremunCap=dfmulti.groupby(["timestamp"])[["returnCap"]].max().sum()
int(yearlyremunAct)+int(yearlyremunCap)

4300541

In [36]:
print(yearlyremunCap/1000)

returnCap    2407.1216
dtype: float64


In [37]:
print(yearlyremunAct/1000)

returnAct    1893.420722
dtype: float64


In [ ]:
(pd.to_datetime(dfmulti["timestamp"].unique()).day)

In [ ]:
dfmulti = dfmulti.set_index("bidprice in euro/MWh")
fig,(ax,ax1,ax2) = plt.subplots(3,1, sharex=True)
dfmulti.groupby(["timestamp"])[["returnAct"]].idxmax().plot(ax=ax,kind="bar",grid=True,legend=False)
dfmulti.groupby(["timestamp"])[["returnAct","returnCap"]].max().plot(ax=ax1,kind="bar",grid=True,color=["red","green"],legend=False)
dfmulti.groupby(["timestamp"])[["volumeActiv","timeActiv"]].max().plot(ax=ax2,kind="bar",grid=True,legend=False)
ax.set_xticklabels(pd.to_datetime(dfmulti["timestamp"].unique()).week, rotation=90)
ax1.legend(("activation","capacity"))
ax2.legend(("volume","time"))
ax2.set_xlabel("Weeks of the year ")
ax.set_ylabel("Optimal marginal activation \n prices in euro/MWh")
ax1.set_ylabel("Remuneration in euro")
ax2.set_ylabel("Activation percentage")
fig.suptitle("Elia R2+ product optimised bidprices \nfor maximum activation remuneration")


In [ ]:
dfAct = pd.read_pickle("../data/ActivEnergyPriceVol_01-2012_12-2017.pkl")
dfCap = pd.read_pickle("../data/AncilServR1R2_2014-2017.pkl")
fromdate = "2017-01-02"
todate = "2017-01-31"
frequency = 7
bidvolume = 1
dataframeAct = dfAct
dataframeCap = dfCap
variable = True
    
df = dataframeAct
df1 = dataframeCap
    
fromdatepd=pd.to_datetime(fromdate,format="%Y-%m-%d")
todatepd=pd.to_datetime(todate,format="%Y-%m-%d")
todatepd = todatepd + relativedelta(days=1)
date=fromdatepd
date1=fromdatepd+relativedelta(days=frequency)-relativedelta(seconds=1)

dfR2plus = pd.DataFrame(columns=["timestamp","bidprice in euro/MWh","number of quarters","timeActiv","volumeActiv","return"])

while (todatepd-relativedelta(days=frequency)) >= date:
    print(date)
    print(date1)
    temp = R2plusSingleOptimum(str(date),str(date1),bidvolume,df,df1,True,"no")
    temp.reset_index(inplace=True)
    temp ["timestamp"] = date
    dfR2plus = dfR2plus.append(temp,ignore_index=True)
    date = date + relativedelta(days=frequency)
    date1 = date1 + relativedelta(days=frequency)
    

In [ ]:
dfR2plus=pd.read_pickle("../data/temptest.pkl")
fig,(ax,ax1) = plt.subplots(2,1)
dfR2plus.groupby(["timestamp"])[["returnAct"]].idxmax().plot(ax=ax,style=".-",grid=True)
dfR2plus.groupby(["timestamp"])[["returnAct","returnCap"]].max().plot(ax=ax1,style=".-",grid=True)

In [ ]:
dfmulti=R2plusMultiTimeMultiVolume("2017-02-01","2017-02-28",7,1,26,5,df)

In [ ]:
fromdate = "2017-10-01"
todate = "2017-10-14"
bidvolume = 100
dataframe1 = df
dataframe2 = df1
plotting = "plotting"

df = dataframe1
df1 = dataframe2
    
dfmaskR2plusresample=df1.loc[(df1["Reserve Type"] == "R2") & (df1["Service Type"] == "Upward")].resample("15T").mean().ffill()
dfmaskR2plusresample=dfmaskR2plusresample.rename(columns={"Total Contracted Volume[MW]":"total contracted volume in MW","Average Price[euro/Mw/h]":"average availability price in euro/MW/h","Total Offered Volume[MW]":"total offered volume in MW"})
dfmaskR2plusresample=dfmaskR2plusresample.drop("Forecasted Average Price[euro/Mw/h]",axis=1)
df = df[["R2+ in euro/MWh","R2+ in MW"]]
df = df.join(dfmaskR2plusresample,how="inner")
df=df.rename(columns={"R2+ in euro/MWh":"marginal activation price in euro/MWh","R2+ in MW":"activated volume in MW"})
    
mask=df.loc[(df.index >= fromdate) & (df.index <= todate)]
dfR2plus= pd.DataFrame(columns=["bidprice in euro/MWh","number of quarters","timeActiv","volumeActiv","returnAct","returnCapAndAct"])
looprange = sorted(mask["marginal activation price in euro/MWh"].unique()[np.logical_not(np.isnan(mask["marginal activation price in euro/MWh"].unique()))])
looprange = [float(Decimal("%.2f" % e)) for e in looprange]
        
for i in looprange:
    mask['euro/MWh'] = mask.apply(lambda row: (i
                                        if i<= row["marginal activation price in euro/MWh"]
                                        else 0),axis=1)
    mask['volumeActiv'] = mask.apply((lambda row: (row["activated volume in MW"]/row["total contracted volume in MW"])
                                        if i<= row["marginal activation price in euro/MWh"]
                                        else 0),axis=1)
        
        
    dfR2plus=dfR2plus.append({"bidprice in euro/MWh":i,
                                  "number of quarters":(mask[mask["marginal activation price in euro/MWh"]>=i]["marginal activation price in euro/MWh"].count()),
                                  "timeActiv":(mask[mask["marginal activation price in euro/MWh"]>=i]["marginal activation price in euro/MWh"].count()/len(mask)),
                                  "volumeActiv":(mask.apply(lambda row: (row['volumeActiv']),axis=1).mean()),
                                  "returnAct":(mask.apply(lambda row: ((((bidvolume/row["total contracted volume in MW"])*(row["activated volume in MW"]/4))*row["euro/MWh"])),axis=1).sum()),
                                  "returnCapAndAct":(mask.apply(lambda row: ((((bidvolume/row["total contracted volume in MW"])*(row["activated volume in MW"]/4))*row["euro/MWh"])+(row["average availability price in euro/MW/h"]*bidvolume/4)),axis=1).sum())},
                                 ignore_index=True)

dfR2plus.set_index("bidprice in euro/MWh",inplace=True)

if (plotting=="plotting"):
    fig,(ax,ax1) = plt.subplots(2,1)
    (dfR2plus["returnAct"]/1000).plot(ax=ax, grid=True)
    (dfR2plus["returnCapAndAct"]/1000).plot(ax=ax, grid=True)
    (dfR2plus["volumeActiv"]*100).plot(ax=ax1, grid=True)
    (dfR2plus["timeActiv"]*100).plot(ax=ax1, grid=True)

    ax.set_xlabel("Marginal activation prices in euro/MWh")
    ax1.set_xlabel("Marginal activation prices in euro/MWh")
    ax.set_ylabel("Cummulative activation remuneration \n for selected period in thousands of euro")
    ax1.set_ylabel("Activation percentage")
    fig.suptitle("Elia R2+ product optimal bidprice\n averaged for the period " + fromdate + " to " + todate +"\nfor a volume of " + str(bidvolume) + "MW")

    ax.plot(dfR2plus["returnAct"].idxmax(), (dfR2plus["returnAct"]/1000).max(), 'or')
    ax.annotate(xy=(dfR2plus["returnAct"].idxmax(), (dfR2plus["returnAct"]/1000).max()),s=(str(round(dfR2plus["returnAct"].idxmax(),1)),str(round((dfR2plus["returnAct"]/1000).max(),1))))
    ax1.plot(dfR2plus["returnAct"].idxmax(), dfR2plus.loc[dfR2plus["returnAct"].idxmax()]["timeActiv"]*100, 'or')
    ax1.annotate(xy=(dfR2plus["returnAct"].idxmax(), dfR2plus.loc[dfR2plus["returnAct"].idxmax()]["timeActiv"]*100), s=(str(round(dfR2plus["returnAct"].idxmax(),1)),str(round((dfR2plus.loc[dfR2plus["returnAct"].idxmax()]["timeActiv"]*100),1))))
    ax1.plot(dfR2plus["returnAct"].idxmax(), dfR2plus.loc[dfR2plus["returnAct"].idxmax()]["volumeActiv"]*100, 'or')
    ax1.annotate(xy=(dfR2plus["returnAct"].idxmax(), dfR2plus.loc[dfR2plus["returnAct"].idxmax()]["volumeActiv"]*100), s=(str(round(dfR2plus["returnAct"].idxmax(),1)),str(round((dfR2plus.loc[dfR2plus["returnAct"].idxmax()]["volumeActiv"]*100),1))))
    ax1.legend(("Volume", "Time"))

In [ ]:
def R2plusSingleOptimum1(fromdate,todate,bidvolume,dataframe1,dataframe2,plotting):
    """
    Create dataframe with:
    the bidprice in euro/MWh
    the return in euro for the selected period
    the activation time in percentage of the selected period
    the average activated volume of the bidvolume  
    
    Variable steps are used, based on the real marginal prices occuring during the selected period.
    
    Parameters
    ----------
    fromdate : string
        date in the format of 'YYYY-MM-DD', e.g., '2017-10-22' is the 22th of October 2017
    todate : string
        date in the format of 'YYYY-MM-DD', e.g., '2017-10-22' is the 22th of October 2017
    bidvolume : int
        the volume of power for which is bidded (in MW)
    dataframe1 : dataframe
        dataframe made by executing the "FetchAndCombine_Elia_ActivEnergy" method. Contains all the prices and volumes of the activated ancillary services.
    dataframe2 : dataframe
        dataframe containing the avereged prices for the standby remuneration for the ancillary services.
    plotting : string
        if you which to see the data in a plot, type "plotting"
        
    returns
    -------
    DataFrame
        Processed dataframe.
    plot
        if plotting is selected
    """
    df = dataframe1
    df1 = dataframe2
    
    dfmaskR2plusresample=df1.loc[(df1["Reserve Type"] == "R2") & (df1["Service Type"] == "Upward")].resample("15T").mean().ffill()
    dfmaskR2plusresample=dfmaskR2plusresample.rename(columns={"Total Contracted Volume[MW]":"total contracted volume in MW","Average Price[€/Mw/h]":"average availability price in euro/MW/h","Total Offered Volume[MW]":"total offered volume in MW"})
    dfmaskR2plusresample=dfmaskR2plusresample.drop("Forecasted Average Price[€/Mw/h]",axis=1)
    df = df[["R2+ in euro/MWh","R2+ in MW"]]
    df = df.join(dfmaskR2plusresample,how="inner")
    df=df.rename(columns={"R2+ in euro/MWh":"marginal activation price in euro/MWh","R2+ in MW":"activated volume in MW"})
    
    mask=df.loc[(df.index >= fromdate) & (df.index <= todate)]
    dfR2plus= pd.DataFrame(columns=["bidprice in euro/MWh","number of quarters","timeActiv","volumeActiv","returnAct","returnCapAndAct"])
    looprange = sorted(mask["marginal activation price in euro/MWh"].unique()[np.logical_not(np.isnan(mask["marginal activation price in euro/MWh"].unique()))])
    looprange = [float(Decimal("%.2f" % e)) for e in looprange]
        
    for i in looprange:
        mask['euro/MWh'] = mask.apply(lambda row: (i
                                        if i<= row["marginal activation price in euro/MWh"]
                                        else 0),axis=1)
        mask['volumeActiv'] = mask.apply((lambda row: (row["activated volume in MW"]/row["total contracted volume in MW"])
                                        if i<= row["marginal activation price in euro/MWh"]
                                        else 0),axis=1)
        
        
        dfR2plus=dfR2plus.append({"bidprice in euro/MWh":i,
                                  "number of quarters":(mask[mask["marginal activation price in euro/MWh"]>=i]["marginal activation price in euro/MWh"].count()),
                                  "timeActiv":(mask[mask["marginal activation price in euro/MWh"]>=i]["marginal activation price in euro/MWh"].count()/len(mask)),
                                  "volumeActiv":(mask.apply(lambda row: (row['volumeActiv']),axis=1).mean()),
                                  "returnAct":(mask.apply(lambda row: ((((bidvolume/row["total contracted volume in MW"])*(row["activated volume in MW"]/4))*row["euro/MWh"])),axis=1).sum()),
                                  "returnCapAndAct":(mask.apply(lambda row: ((((bidvolume/row["total contracted volume in MW"])*(row["activated volume in MW"]/4))*row["euro/MWh"])+(row["average availability price in euro/MW/h"]*bidvolume/4)),axis=1).sum())},
                                 ignore_index=True)

    dfR2plus.set_index("bidprice in euro/MWh",inplace=True)

    if (plotting=="plotting"):
        fig,(ax,ax1) = plt.subplots(2,1)
        (dfR2plus["returnAct"]/1000).plot(ax=ax, grid=True)
        (dfR2plus["returnCapAndAct"]/1000).plot(ax=ax, grid=True)
        (dfR2plus["volumeActiv"]*100).plot(ax=ax1, grid=True)
        (dfR2plus["timeActiv"]*100).plot(ax=ax1, grid=True)

        ax.set_xlabel("Marginal activation prices in euro/MWh")
        ax1.set_xlabel("Marginal activation prices in euro/MWh")
        ax.set_ylabel("Cummulative activation remuneration \n for selected period in thousands of euro")
        ax1.set_ylabel("Activation percentage")
        fig.suptitle("Elia R2+ product optimal bidprice\n averaged for the period " + fromdate + " to " + todate +"\nfor a volume of " + str(bidvolume) + "MW")

        ax.plot(dfR2plus["returnAct"].idxmax(), (dfR2plus["returnAct"]/1000).max(), 'or')
        ax.annotate(xy=(dfR2plus["returnAct"].idxmax(), (dfR2plus["returnAct"]/1000).max()),s=(str(round(dfR2plus["returnAct"].idxmax(),1)),str(round((dfR2plus["returnAct"]/1000).max(),1))))
        ax1.plot(dfR2plus["returnAct"].idxmax(), dfR2plus.loc[dfR2plus["returnAct"].idxmax()]["timeActiv"]*100, 'or')
        ax1.annotate(xy=(dfR2plus["returnAct"].idxmax(), dfR2plus.loc[dfR2plus["returnAct"].idxmax()]["timeActiv"]*100), s=(str(round(dfR2plus["returnAct"].idxmax(),1)),str(round((dfR2plus.loc[dfR2plus["returnAct"].idxmax()]["timeActiv"]*100),1))))
        ax1.plot(dfR2plus["returnAct"].idxmax(), dfR2plus.loc[dfR2plus["returnAct"].idxmax()]["volumeActiv"]*100, 'or')
        ax1.annotate(xy=(dfR2plus["returnAct"].idxmax(), dfR2plus.loc[dfR2plus["returnAct"].idxmax()]["volumeActiv"]*100), s=(str(round(dfR2plus["returnAct"].idxmax(),1)),str(round((dfR2plus.loc[dfR2plus["returnAct"].idxmax()]["volumeActiv"]*100),1))))
        ax1.legend(("Volume", "Time"))
        
    return dfR2plus

In [ ]:
#calculate for multiple time and volume
fromvolume=1 #in MW
tovolume=81 # in MW
blocksize=10 #in MW
fromdate="2017-05-01"
todate="2017-05-04"
frequency= 2 #in days
#dfR2pluss = pd.DataFrame()
#dfR2pluss = pd.DataFrame()

fromdatepd=pd.to_datetime(fromdate,format="%Y-%m-%d")
todatepd=pd.to_datetime(todate,format="%Y-%m-%d")
todatepd = todatepd + relativedelta(days=1)
date=fromdatepd
date1=fromdatepd+relativedelta(days=frequency)

dfR2plus = pd.DataFrame(columns=["timestamp","contractedPower in MW","bidprice in euro/MWh","number of quarters","timeActiv","volumeActiv","return"])

while (todatepd-relativedelta(days=frequency)) >= date:
    for i in np.arange(fromvolume,tovolume+1,blocksize):
        temp = R2plusSingleOptimum(str(date),str(date1),i,df,"no")
        temp.reset_index(inplace=True)
        temp ["contractedPower in MW"] = i
        temp ["timestamp"] = date
        dfR2plus = dfR2plus.append(temp,ignore_index=True)
    date = date + relativedelta(days=frequency)
    date1 = date1 + relativedelta(days=frequency)
    
dfR2plus.set_index("bidprice in euro/MWh", inplace=True)
dfR2pluss=dfR2plus.groupby(["timestamp", "contractedPower in MW"])["return"].idxmax().reset_index()
dfR2plusss=dfR2plus.groupby(["timestamp", "contractedPower in MW"])["return"].max().reset_index()
dfR2plusr = dfR2pluss.merge(dfR2plusss,on=["timestamp","contractedPower in MW"])
dfR2plusr=dfR2plusr.rename(columns={"return_x":"bidprice in euro/MWh","return_y":"return"})

In [ ]:
dfR2plusr

In [ ]:
dfR2plusr=dfR2plusr.reset_index()
dfR2plusr=dfR2plusr.set_index("contractedPower in MW")

In [ ]:
dfR2plusr["return"].plot(grid=True)

In [ ]:
#plot for multiple bidvolumes, same period
dfR2plusMWrange= pd.DataFrame(columns=["contractedPower in MW","bidprice in euro/MWh","number of quarters","timeActiv","volumeActiv","return"])
for i in range (5,50):
    temp = remunR2plus("2017-10-01","2017-10-14",i,df,"no")
    temp.reset_index(inplace=True)
    temp ["contractedPower in MW"] = i
    dfR2plusMWrange = dfR2plusMWrange.append(temp,ignore_index=True)
dfR2plusMWrange.set_index("bidprice in euro/MWh", inplace=True)
picklename = "..\data\dfR2plusMWrange_20171001_20171014.pkl"
dfR2plusMWrange.to_pickle(picklename)

In [ ]:
mask=dfR2plusMWrange.loc[(dfR2plusMWrange["contractedPower in MW"] >= 5) & (dfR2plusMWrange["contractedPower in MW"] <= 50)]
mask.groupby(["contractedPower in MW"])["return"].max().plot()

In [ ]:
dfR2plusMWrange = pd.read_pickle("../data/dfR2plusMWrange_20171001_20171014.pkl")
mask=dfR2plusMWrange.loc[(dfR2plusMWrange["contractedPower in MW"] >= 5) & (dfR2plusMWrange["contractedPower in MW"] <= 45)]
fig, ax = plt.subplots()
mask.groupby(["contractedPower in MW"])["return"].plot(figsize=(16,6),legend=True, ax=ax,kind="line", style='.-',grid=True)


In [ ]:
#create mask for all plotting
fromdate = "2017-06-01"
todate = "2017-06-10"
mask=df.loc[(df.index >= fromdate) & (df.index <= todate)]

In [ ]:
#timeplot of R2+; separate plots for volume and price
fig, (ax,ax1) = plt.subplots(2,1)
mask[["R2+ in euro/MWh"]].plot(ax=ax, grid=True, style=".")
mask[["R2+ in MW"]].plot(ax=ax1, grid=True)

In [ ]:
#scatterplot of R2+; activated volumes (MW) versus price in euro/MWh
fig,ax = plt.subplots()
ax.scatter(mask['R2+ in euro/MWh'], mask['R2+ in MW'], marker = '.', alpha = 0.2)
fig.show()
ax.set_xlabel("R2+ in euro/MWh")
ax.set_ylabel("R2+ in MW activated")
fig.suptitle("Elia R2+ product\n" + fromdate + " to " + todate)

In [ ]:
#barplot of R2+ volumes and prices averaged by hour of the day
fig, (ax,ax1) = plt.subplots(1,2)
mask.groupby(mask.index.hour)["R2+ in euro/MWh"].mean().plot(figsize=(16,6),legend=False, ax=ax,kind="line",grid=True,style=".-")
mask.groupby(mask.index.hour)["R2+ in MW"].mean().plot(figsize=(16,6),legend=False, ax=ax1,kind="line",grid=True,style=".-")

ax.set_xlabel("Hours of the day")
ax.set_ylabel("R2+ in euro/MWh")
ax1.set_xlabel("Hours of the day")
ax1.set_ylabel("R2+ in MW activated")
fig.suptitle("Elia R2+ product\n" + fromdate + " to " + todate)

In [ ]:
#barplot of R2+ volumes and prices averaged by weekday
fig, (ax,ax1) = plt.subplots(1,2)
mask.groupby(mask.index.weekday)["R2+ in euro/MWh"].mean().plot(figsize=(16,6),legend=False, ax=ax,kind="bar",grid=True)
mask.groupby(mask.index.weekday)["R2+ in MW"].mean().plot(figsize=(16,6),legend=False, ax=ax1,kind="bar",grid=True)
ax.set_xlabel("Days of the week (0 = monday)")
ax.set_ylabel("R2+ in euro/MWh")
ax1.set_xlabel("Days of the week (0 = monday)")
ax1.set_ylabel("R2+ in MW activated")
fig.suptitle("Elia R2+ product\n" + fromdate + " to " + todate)

In [ ]:
#3d figuur
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xs=mask.index.hour,ys=mask["R2+ in MW"],zs=mask["R2+ in euro/MWh"],depthshade=True)
ax.set_xlabel("Hours of the day")
ax.set_ylabel("R2+ in MW")
ax.set_zlabel("R2+ in euro/MWh")

In [ ]:
#R2+ timeplot for extreme activated volumes
fig, (ax,ax1) = plt.subplots(2,1)
mask[["R2+ in euro/MWh"]].loc[mask["R2+ in MW"] < 2].plot(ax=ax, grid=True, style=".",legend=False)
mask[["R2+ in MW"]].loc[mask["R2+ in MW"] < 2].plot(ax=ax1, grid=True,style=".",legend=False)
mask[["R2+ in euro/MWh"]].loc[mask["R2+ in MW"] > 120].plot(ax=ax, grid=True, style=".",legend=False)
mask[["R2+ in MW"]].loc[mask["R2+ in MW"] > 120].plot(ax=ax1, grid=True,style=".",legend=False)
ax.set_xlabel("")
ax.set_ylabel("R2+ in euro/MWh")
ax1.set_xlabel("")
ax1.set_ylabel("R2+ in MW activated")

In [ ]:
#plots the graphs of bidprice in euro/MWh versus the activation payment for the selected period and bidvolume in k€/MW ánd
#the graphs of the bidprice in euro/MWh versus the % of time activation is needed. Also the optimum bidprice, as well as the total maximum payment, for the selected period is shown as red dot.
#One graph per timeperiod is plotted, a timeframe can be selected by adapting the "fromdate" and "todate" parameters. The timeperiod, over which the mean is calculated, can be changed by the parameter "frequency"

#parameters to adapt:
fromdate = "2017-01-01"
todate = "2017-03-01"
bidvolume = 20
frequency = 1 # in months

#code
legendstrings = []
legendstrings2 = []
fromdatepd=pd.to_datetime(fromdate,format="%Y-%m-%d")
todatepd=pd.to_datetime(todate,format="%Y-%m-%d")
date=fromdatepd
fig,(ax,ax1) = plt.subplots(2,1)
while (date < todatepd):
    mask=df.loc[(df.index >= date) & (df.index <= date + relativedelta(months=frequency))]
    dfR2plus= pd.DataFrame(columns=["number of quarters","percentage of time","Return","volumeActiv"])
    startR2plus = math.floor(mask["R2+ in euro/MWh"].min())
    stopR2plus = math.ceil(mask["R2+ in euro/MWh"].max())

    for i in range (startR2plus*10,stopR2plus*10):
        dfR2plus=dfR2plus.append({"# of quarters":(mask[mask["R2+ in euro/MWh"]>i/10]["R2+ in euro/MWh"].count()),"percentage of time":(mask[mask["R2+ in euro/MWh"]>i/10]["R2+ in euro/MWh"].count()/len(mask))}, ignore_index=True)
  
    dfR2plus.index.name = "euro/MWh"
    dfR2plus.index = (dfR2plus.index/10)+startR2plus
    
    for i in range (startR2plus*10,stopR2plus*10):    
        mask['euro/MWh'] = mask.apply(lambda row: (row["R2+ in euro/MWh"]/4
                                        if i/10<= row["R2+ in euro/MWh"]
                                        else np.nan),axis=1)
        dfR2plus["Return"][i/10] = mask.apply(lambda row: (row['R2+ in MW']*row['euro/MWh']
                                        if row['R2+ in MW']<bidvolume
                                        else bidvolume*row['euro/MWh']),axis=1).sum()
        
        mask['volumeActiv'] = mask.apply(lambda row: ((1*row['euro/MWh'])/row['euro/MWh']
                                        if row["R2+ in MW"]>bidvolume
                                        else ((row["R2+ in MW"]/bidvolume)*row['euro/MWh'])/row['euro/MWh']),axis=1)
        dfR2plus["volumeActiv"][i/10] = mask.apply(lambda row: (row['volumeActiv']),axis=1).mean()

    dfR2plus["Return"]=pd.DataFrame(dfR2plus["Return"], dtype='float')    
    
    (dfR2plus["Return"]/1000).plot(ax=ax, grid=True,)
    (dfR2plus["volumeActiv"]*100).plot(ax=ax1, grid=True)
    (dfR2plus["percentage of time"]*100).plot(ax=ax1, grid=True)
    
    ax.plot(dfR2plus["Return"].idxmax(), (dfR2plus["Return"]/1000).max(), 'or', label='_nolegend_')
    ax.annotate(xy=(dfR2plus["Return"].idxmax(), (dfR2plus["Return"]/1000).max()),s=(str(round(dfR2plus["Return"].idxmax(),1)),str(round((dfR2plus["Return"]/1000).max(),1))))
    ax1.plot(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["percentage of time"]*100, 'or', label='_nolegend_')
    ax1.annotate(xy=(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["percentage of time"]*100), s=(str(round(dfR2plus["Return"].idxmax(),1)),str(round((dfR2plus.loc[dfR2plus["Return"].idxmax()]["percentage of time"]*100),1))))
    legendstrings.append(str(date.year)+"/"+str(date.month)+" - "+str((date+relativedelta(months=frequency)).year)+"/"+str((date+relativedelta(months=frequency)).month))
    legendstrings2.append(str(date.year)+"/"+str(date.month)+" - "+str((date+relativedelta(months=frequency)).year)+"/"+str((date+relativedelta(months=frequency)).month) + " volume")
    legendstrings2.append(str(date.year)+"/"+str(date.month)+" - "+str((date+relativedelta(months=frequency)).year)+"/"+str((date+relativedelta(months=frequency)).month) + " time")
    date = date + relativedelta(months=frequency) 
    
ax.legend(labels=legendstrings)
ax1.legend(labels=legendstrings2)
ax.set_xlabel("Marginal activation prices in euro/MWh")
ax1.set_xlabel("Marginal activation prices in euro/MWh")
ax.set_ylabel("Cummulative activation remuneration \n for selected period in k\u20ac/MW")
ax1.set_ylabel("Activation percentage")
fig.suptitle("Elia R2+ product optimal bidprice\n averaged for the chosen periods for a volume of " + str(bidvolume) + "MW")


In [ ]:
#parameters to adapt:
fromdate = "2016-01-01"
todate = "2016-12-31"
frequency = 1 # in days

dfopti=pd.DataFrame(columns=["timestamp","optimal bid price in euro/MW"])

#code
fromdatepd=pd.to_datetime(fromdate,format="%Y-%m-%d")
todatepd=pd.to_datetime(todate,format="%Y-%m-%d")
date=fromdatepd
fig,ax = plt.subplots(1,1)
while (date < todatepd):
    mask=df.loc[(df.index >= date) & (df.index <= date + relativedelta(days=frequency))]
    
    dfR2plus= pd.DataFrame(columns=["# of quarters","% of time"])

    startR2plus = math.floor(mask["R2+ in euro/MWh"].min())
    stopR2plus = math.ceil(mask["R2+ in euro/MWh"].max())


    for i in range (startR2plus*10,stopR2plus*10):
        dfR2plus=dfR2plus.append({"# of quarters":(mask[mask["R2+ in euro/MWh"]>i/10]["R2+ in euro/MWh"].count()),"% of time":(mask[mask["R2+ in euro/MWh"]>i/10]["R2+ in euro/MWh"].count()/len(mask))}, ignore_index=True)
        
    dfR2plus.index.name = "euro/MWh"
    dfR2plus.index = (dfR2plus.index/10)+startR2plus
    dfR2plus["Return"] = (dfR2plus.index)/4 * dfR2plus["# of quarters"]

    dfopti=dfopti.append({"timestamp":(date),"optimal bid price in euro/MW":(dfR2plus["Return"].idxmax())},ignore_index=True)
    dfopti.set_index("timestamp",inplace=True)
    dfopti.plot(ax=ax, grid=True, style=".-")
    ax.legend_.remove()
    date = date + relativedelta(days=frequency) 

ax.set_ylabel("Optimal bid price in euro/MWh")

In [ ]:
#create dataframe with the R2+ product prices within a specific timeframe, taking into account the volumes
fromdate = "2016-01-01"
todate = "2016-12-31"
bidvolume = 1

mask=df.loc[(df.index >= fromdate) & (df.index <= todate)]

for i in len(mask):
    if mask.loc[mask["R2+ in MW"] < bidvolume, 'b']: 
        mask["remunR2+ in euro/MWh"] = bidvolume*mask["R2+ in euro/MWh"]/4
    else:
        mask["remunR2+ in euro/MWh"] = mask["R2+ in MW"]*mask["R2+ in euro/MWh"]/4
    


dfR2plus= pd.DataFrame(columns=["# of quarters","% of time"])
dfR2min= pd.DataFrame(columns=["# of quarters","% of time"])
startR2plus = math.floor(mask["R2+ in euro/MWh"].min())
stopR2plus = math.ceil(mask["R2+ in euro/MWh"].max())
startR2min = math.floor(mask["R2- in euro/MWh"].min())
stopR2min = math.ceil(mask["R2- in euro/MWh"].max())

for i in range (startR2plus*10,stopR2plus*10):
    dfR2plus=dfR2plus.append({"# of quarters":(mask[mask["R2+ in euro/MWh"]>i/10]["R2+ in euro/MWh"].count()),"% of time":(mask[mask["R2+ in euro/MWh"]>i/10]["R2+ in euro/MWh"].count()/len(mask))}, ignore_index=True)
for i in range (startR2min*10,stopR2min*10):
    dfR2min=dfR2min.append({"# of quarters":(mask[mask["R2- in euro/MWh"]<i/10]["R2- in euro/MWh"].count()),"% of time":(mask[mask["R2- in euro/MWh"]<i/10]["R2- in euro/MWh"].count()/len(mask))}, ignore_index=True)
    
dfR2plus.index.name = "euro/MWh"
dfR2plus.index = (dfR2plus.index/10)+startR2plus
dfR2plus["Return"] = (dfR2plus.index)/4 * dfR2plus["# of quarters"]

dfR2min.index.name = "euro/MWh"
dfR2min.index = (dfR2min.index/10)+startR2min
dfR2min["Return"] = -(dfR2min.index)/4 * dfR2min["# of quarters"]

#plot the R2+ product with the maximum return and maximum remuneration for the chosen period

fig,(ax,ax1) = plt.subplots(2,1)
(dfR2plus["Return"]/1000).plot(ax=ax, grid=True,)
(dfR2min["Return"]/1000).plot(ax=ax, grid=True,)
(dfR2plus["% of time"]*100).plot(ax=ax1, grid=True)
(dfR2min["% of time"]*100).plot(ax=ax1, grid=True)
fig.legend(("R2 upwards","R2 downwards"))

ax.set_xlabel("bidprice in € per MWh")
ax1.set_xlabel("bidprice in € per MWh")
ax.set_ylabel("activation payment for the selected period in k€/MW")
ax1.set_ylabel("% of time activation needed")

ax.plot(dfR2plus["Return"].idxmax(), (dfR2plus["Return"]/1000).max(), 'or')
ax.annotate(xy=(dfR2plus["Return"].idxmax(), (dfR2plus["Return"]/1000).max()),s=(round(dfR2plus["Return"].idxmax()),round(((dfR2plus["Return"]/1000).max()))))
ax1.plot(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["% of time"]*100, 'or')
ax1.annotate(xy=(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["% of time"]*100), s=(round(dfR2plus["Return"].idxmax()),round((dfR2plus.loc[dfR2plus["Return"].idxmax()]["% of time"]*100))))

In [ ]:
bidvolume = 1
#bidprice = 39.85
fromdate = "2016-01-01"
todate = "2016-01-31"
mask=df.loc[(df.index >= fromdate) & (df.index <= todate)]

for bidprice in range (30,72):
    mask['euro/MWh'] = mask.apply(lambda row: (row["R2+ in euro/MWh"]/4
                                                  if bidprice<= row["R2+ in euro/MWh"]
                                                  else np.nan ),axis=1)
    print (mask.apply(lambda row: (row['R2+ in MW']*row['euro/MWh']
                                                   if row['R2+ in MW']<bidvolume
                                                   else bidvolume*row['euro/MWh']),axis=1).sum())

In [ ]:
bidvolume = 10
bidprice = 40
fromdate = "2016-01-01"
todate = "2016-01-31"
mask=df.loc[(df.index >= fromdate) & (df.index <= todate)]
mask['euro/MWh'] = mask.apply(lambda row: (row["R2+ in euro/MWh"]/4
                                if bidprice<= row["R2+ in euro/MWh"]
                                else np.nan ),axis=1)
print(mask.apply(lambda row: (row['R2+ in MW']*row['euro/MWh']
                            if row['R2+ in MW']<bidvolume
                            else bidvolume*row['euro/MWh']),axis=1).sum())


In [ ]:
# create dataframe with the R2+ product prices within a specific timeframe (with fixed steps)
fromdate = "2017-10-01"
todate = "2017-10-10"
bidvolume = 1

mask=df.loc[(df.index >= fromdate) & (df.index <= todate)]
dfR2plus= pd.DataFrame(columns=["# of quarters","% of time","Return","volumeActiv"])
dfBidsplus= pd.DataFrame(columns=["# of quarters","% of time","Return","volumeActiv"])
startR2plus = math.floor(mask["R2+ in euro/MWh"].min())-10
stopR2plus = math.ceil(mask["R2+ in euro/MWh"].max())
startBidsplus = math.floor(mask["Bids+ in euro/MWh"].min())
stopBidsplus = math.ceil(mask["Bids+ in euro/MWh"].max())

for i in range (startR2plus*10,stopR2plus*10):
    dfR2plus=dfR2plus.append({"# of quarters":(mask[mask["R2+ in euro/MWh"]>=i/10]["R2+ in euro/MWh"].count()),"% of time":(mask[mask["R2+ in euro/MWh"]>=i/10]["R2+ in euro/MWh"].count()/len(mask))}, ignore_index=True)
for i in range (startBidsplus*10,stopBidsplus*10):
    dfBidsplus=dfBidsplus.append({"# of quarters":(mask[mask["Bids+ in euro/MWh"]>=i/10]["Bids+ in euro/MWh"].count()),"% of time":(mask[mask["Bids+ in euro/MWh"]>=i/10]["Bids+ in euro/MWh"].count()/len(mask))}, ignore_index=True)
    
dfR2plus.index.name = "euro/MWh"
dfR2plus.index = (dfR2plus.index/10)+startR2plus
dfBidsplus.index.name = "euro/MWh"
dfBidsplus.index = (dfBidsplus.index/10)+startBidsplus


for i in range (startR2plus*10,stopR2plus*10):    
    mask['euro/MWh'] = mask.apply(lambda row: ((i/10)/4
                                    if i/10<= row["R2+ in euro/MWh"]
                                    else 0),axis=1)
    dfR2plus["Return"][i/10] = mask.apply(lambda row: (row['R2+ in MW']*row['euro/MWh']
                                    if row['R2+ in MW']<bidvolume
                                    else bidvolume*row['euro/MWh']),axis=1).sum()
    
    mask['volumeActiv'] = mask.apply(lambda row: ((1*row['euro/MWh'])/row['euro/MWh']
                                    if row["R2+ in MW"]>bidvolume
                                    else ((row["R2+ in MW"]/bidvolume)*row['euro/MWh'])/row['euro/MWh']),axis=1)
    dfR2plus["volumeActiv"][i/10] = mask.apply(lambda row: (row['volumeActiv']),axis=1).mean()

for i in range (startBidsplus*10,stopBidsplus*10):    
    mask['euro/MWh'] = mask.apply(lambda row: ((i/10)/4
                                    if i/10<= row["Bids+ in euro/MWh"]
                                    else 0),axis=1)
    dfBidsplus["Return"][i/10] = mask.apply(lambda row: (row['Bids+ in MW']*row['euro/MWh']
                                    if row['Bids+ in MW']<bidvolume
                                    else bidvolume*row['euro/MWh']),axis=1).sum()
    
    mask['volumeActiv'] = mask.apply(lambda row: ((1*row['euro/MWh'])/row['euro/MWh']
                                    if row["Bids+ in MW"]>bidvolume
                                    else ((row["Bids+ in MW"]/bidvolume)*row['euro/MWh'])/row['euro/MWh']),axis=1)
    dfBidsplus["volumeActiv"][i/10] = mask.apply(lambda row: (row['volumeActiv']),axis=1).mean()
    
    
#plot the R2+ product with the maximum return and maximum remuneration for the chosen period

fig,(ax,ax1) = plt.subplots(2,1)
(dfR2plus["Return"]/1000).plot(ax=ax, grid=True)
(dfBidsplus["Return"]/1000).plot(ax=ax, grid=True)
(dfR2plus["volumeActiv"]*100).plot(ax=ax1, grid=True)
(dfR2plus["% of time"]*100).plot(ax=ax1, grid=True)

ax.set_xlabel("Marginal activation prices in €/MWh")
ax1.set_xlabel("Marginal activation prices in €/MWh")
ax.set_ylabel("Cummulative activation remuneration \n for selected period in k€/MW")
ax1.set_ylabel("Activation %")
fig.suptitle("Elia R2+ product optimal bidprice\n averaged for the period " + fromdate + " to " + todate +"\nfor a volume of " + str(bidvolume) + "MW")

ax.plot(dfR2plus["Return"].idxmax(), (dfR2plus["Return"]/1000).max(), 'or')
ax.annotate(xy=(dfR2plus["Return"].idxmax(), (dfR2plus["Return"]/1000).max()),s=(str(round(dfR2plus["Return"].idxmax(),1)),str(round((dfR2plus["Return"]/1000).max(),1))))
ax1.plot(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["% of time"]*100, 'or')
ax1.annotate(xy=(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["% of time"]*100), s=(str(round(dfR2plus["Return"].idxmax(),1)),str(round((dfR2plus.loc[dfR2plus["Return"].idxmax()]["% of time"]*100),1))))
ax1.plot(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["volumeActiv"]*100, 'or')
ax1.annotate(xy=(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["volumeActiv"]*100), s=(str(round(dfR2plus["Return"].idxmax(),1)),str(round((dfR2plus.loc[dfR2plus["Return"].idxmax()]["volumeActiv"]*100),1))))
ax1.legend(("Volume", "Time"))

In [ ]:
#create dataframe with the R2+ product prices within a specific timeframe (with variable steps, based on the unique prices within the period)
fromdate = "2017-10-01"
todate = "2017-10-20"
bidvolume = 10

mask=df.loc[(df.index >= fromdate) & (df.index <= todate)]
dfR2plus= pd.DataFrame(columns=["# of quarters","% of time","Return","volumeActiv"])
#dfBidsplus= pd.DataFrame(columns=["# of quarters","% of time","Return","volumeActiv"])
startR2plus = math.floor(mask["R2+ in euro/MWh"].min())-10
stopR2plus = math.ceil(mask["R2+ in euro/MWh"].max())
#startBidsplus = math.floor(mask["Bids+ in euro/MWh"].min())
#stopBidsplus = math.ceil(mask["Bids+ in euro/MWh"].max())

for i in range (startR2plus*10,stopR2plus*10):
    dfR2plus=dfR2plus.append({"# of quarters":(mask[mask["R2+ in euro/MWh"]>=i/10]["R2+ in euro/MWh"].count()),"% of time":(mask[mask["R2+ in euro/MWh"]>=i/10]["R2+ in euro/MWh"].count()/len(mask))}, ignore_index=True)
#for i in range (startBidsplus*10,stopBidsplus*10):
#    dfBidsplus=dfBidsplus.append({"# of quarters":(mask[mask["Bids+ in euro/MWh"]>=i/10]["Bids+ in euro/MWh"].count()),"% of time":(mask[mask["Bids+ in euro/MWh"]>=i/10]["Bids+ in euro/MWh"].count()/len(mask))}, ignore_index=True)
    
dfR2plus.index.name = "euro/MWh"
dfR2plus.index = (dfR2plus.index/10)+startR2plus
#dfBidsplus.index.name = "euro/MWh"
#dfBidsplus.index = (dfBidsplus.index/10)+startBidsplus


for i in sorted(mask["R2+ in euro/MWh"].unique()[np.logical_not(np.isnan(mask["R2+ in euro/MWh"].unique()))]):    
    mask['euro/MWh'] = mask.apply(lambda row: ((i)/4
                                    if i<= row["R2+ in euro/MWh"]
                                    else 0),axis=1)
    dfR2plus["Return"][i] = mask.apply(lambda row: (row['R2+ in MW']*row['euro/MWh']
                                    if row['R2+ in MW']<bidvolume
                                    else bidvolume*row['euro/MWh']),axis=1).sum()
    
    mask['volumeActiv'] = mask.apply(lambda row: ((1*row['euro/MWh'])/row['euro/MWh']
                                    if row["R2+ in MW"]>bidvolume
                                    else ((row["R2+ in MW"]/bidvolume)*row['euro/MWh'])/row['euro/MWh']),axis=1)
    dfR2plus["volumeActiv"][i] = mask.apply(lambda row: (row['volumeActiv']),axis=1).mean()

#for i in sorted(mask["Bids+ in euro/MWh"].unique()[np.logical_not(np.isnan(mask["Bids+ in euro/MWh"].unique()))]):
   # mask['euro/MWh'] = mask.apply(lambda row: ((i)/4
   #                                 if i<= row["Bids+ in euro/MWh"]
   #                                 else 0),axis=1)
   # dfBidsplus["Return"][i] = mask.apply(lambda row: (row['Bids+ in MW']*row['euro/MWh']
   #                                 if row['Bids+ in MW']<bidvolume
   #                                 else bidvolume*row['euro/MWh']),axis=1).sum()
   # 
   # mask['volumeActiv'] = mask.apply(lambda row: ((1*row['euro/MWh'])/row['euro/MWh']
   #                                 if row["Bids+ in MW"]>bidvolume
   #                                 else ((row["Bids+ in MW"]/bidvolume)*row['euro/MWh'])/row['euro/MWh']),axis=1)
   # dfBidsplus["volumeActiv"][i] = mask.apply(lambda row: (row['volumeActiv']),axis=1).mean()
    
#plot the R2+ product with the maximum return and maximum remuneration for the chosen period

fig,(ax,ax1) = plt.subplots(2,1)
(dfR2plus["Return"]/1000).plot(ax=ax, grid=True)
#(dfBidsplus["Return"]/1000).plot(ax=ax, grid=True)
(dfR2plus["volumeActiv"]*100).plot(ax=ax1, grid=True)
(dfR2plus["% of time"]*100).plot(ax=ax1, grid=True)

ax.set_xlabel("Marginal activation prices in €/MWh")
ax1.set_xlabel("Marginal activation prices in €/MWh")
ax.set_ylabel("Cummulative activation remuneration \n for selected period in k€/MW")
ax1.set_ylabel("Activation %")
fig.suptitle("Elia R2+ product optimal bidprice\n averaged for the period " + fromdate + " to " + todate +"\nfor a volume of " + str(bidvolume) + "MW")

ax.plot(dfR2plus["Return"].idxmax(), (dfR2plus["Return"]/1000).max(), 'or')
ax.annotate(xy=(dfR2plus["Return"].idxmax(), (dfR2plus["Return"]/1000).max()),s=(str(round(dfR2plus["Return"].idxmax(),1)),str(round((dfR2plus["Return"]/1000).max(),1))))
ax1.plot(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["% of time"]*100, 'or')
ax1.annotate(xy=(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["% of time"]*100), s=(str(round(dfR2plus["Return"].idxmax(),1)),str(round((dfR2plus.loc[dfR2plus["Return"].idxmax()]["% of time"]*100),1))))
ax1.plot(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["volumeActiv"]*100, 'or')
ax1.annotate(xy=(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["volumeActiv"]*100), s=(str(round(dfR2plus["Return"].idxmax(),1)),str(round((dfR2plus.loc[dfR2plus["Return"].idxmax()]["volumeActiv"]*100),1))))
ax1.legend(("Volume", "Time"))

In [ ]:
#create dataframe with the R2+ product prices within a specific timeframe (with variable steps, based on the unique prices within the period)
fromdate = "2017-10-01"
todate = "2017-10-20"
bidvolume = 10

mask=df.loc[(df.index >= fromdate) & (df.index <= todate)]
dfR2plus= pd.DataFrame(columns=["euro/MWh","# of quarters","% of time","Return","volumeActiv"])
looprange = sorted(mask["R2+ in euro/MWh"].unique()[np.logical_not(np.isnan(mask["R2+ in euro/MWh"].unique()))])
looprange = [float(Decimal("%.2f" % e)) for e in looprange]

#startR2plus = math.floor(mask["R2+ in euro/MWh"].min())
#stopR2plus = math.ceil(mask["R2+ in euro/MWh"].max())

#for i in range (startR2plus*10,stopR2plus*10):
    #dfR2plus=dfR2plus.append({"# of quarters":(mask[mask["R2+ in euro/MWh"]>=i/10]["R2+ in euro/MWh"].count()),"% of time":(mask[mask["R2+ in euro/MWh"]>=i/10]["R2+ in euro/MWh"].count()/len(mask))}, ignore_index=True)
    
    
for i in looprange:
    mask['euro/MWh'] = mask.apply(lambda row: ((i)/4
                                    if i<= row["R2+ in euro/MWh"]
                                    else 0),axis=1)
    mask['volumeActiv'] = mask.apply(lambda row: ((1*row['euro/MWh'])/row['euro/MWh']
                                    if row["R2+ in MW"]>bidvolume
                                    else ((row["R2+ in MW"]/bidvolume)*row['euro/MWh'])/row['euro/MWh']),axis=1)
    
    
    dfR2plus=dfR2plus.append({"euro/MWh":i,"# of quarters":(mask[mask["R2+ in euro/MWh"]>=i]["R2+ in euro/MWh"].count()),"% of time":(mask[mask["R2+ in euro/MWh"]>=i]["R2+ in euro/MWh"].count()/len(mask)),"volumeActiv":(mask.apply(lambda row: (row['volumeActiv']),axis=1).mean()),"Return":(mask.apply(lambda row: (row['R2+ in MW']*row['euro/MWh']
                                    if row['R2+ in MW']<bidvolume
                                    else bidvolume*row['euro/MWh']),axis=1).sum())}, ignore_index=True)

dfR2plus.set_index("euro/MWh",inplace=True)

#plotting
fig,(ax,ax1) = plt.subplots(2,1)
(dfR2plus["Return"]/1000).plot(ax=ax, grid=True)
(dfR2plus["volumeActiv"]*100).plot(ax=ax1, grid=True)
(dfR2plus["% of time"]*100).plot(ax=ax1, grid=True)

ax.set_xlabel("Marginal activation prices in €/MWh")
ax1.set_xlabel("Marginal activation prices in €/MWh")
ax.set_ylabel("Cummulative activation remuneration \n for selected period in k€/MW")
ax1.set_ylabel("Activation %")
fig.suptitle("Elia R2+ product optimal bidprice\n averaged for the period " + fromdate + " to " + todate +"\nfor a volume of " + str(bidvolume) + "MW")

ax.plot(dfR2plus["Return"].idxmax(), (dfR2plus["Return"]/1000).max(), 'or')
ax.annotate(xy=(dfR2plus["Return"].idxmax(), (dfR2plus["Return"]/1000).max()),s=(str(round(dfR2plus["Return"].idxmax(),1)),str(round((dfR2plus["Return"]/1000).max(),1))))
ax1.plot(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["% of time"]*100, 'or')
ax1.annotate(xy=(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["% of time"]*100), s=(str(round(dfR2plus["Return"].idxmax(),1)),str(round((dfR2plus.loc[dfR2plus["Return"].idxmax()]["% of time"]*100),1))))
ax1.plot(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["volumeActiv"]*100, 'or')
ax1.annotate(xy=(dfR2plus["Return"].idxmax(), dfR2plus.loc[dfR2plus["Return"].idxmax()]["volumeActiv"]*100), s=(str(round(dfR2plus["Return"].idxmax(),1)),str(round((dfR2plus.loc[dfR2plus["Return"].idxmax()]["volumeActiv"]*100),1))))
ax1.legend(("Volume", "Time"))